In [3]:
import os
import re
import numpy as np
from pathlib import Path
from random import shuffle, seed
from IPython.display import clear_output
import neurox.data.extraction.transformers_extractor as transformers_extractor

In [4]:
path_project = Path(os.getcwd()).parents[1]
path_project = str(path_project)

In [5]:
path_project

'/home/senya/Документы/project'

In [6]:
seed(20)

Конвертируем файл, который получили из Probing_framework (уже получили, не в этом ноутбуке)

In [7]:
class ConvertSample:
    
    
    def __init__(self, path, train_size=2600, test_size=900, shuffle: bool = True): 
        self.shuffle = shuffle
        self.path = path
        self.category = re.search(r'[a-zA-Z]+(?=.csv)', path)[0]
        self.train_size = train_size
        self.test_size = test_size
        

    def read(self) -> list: 
        """
        Reads a file
        Args:
            path: a path to a file
        """
        with open(self.path, encoding="utf-8") as f:
            lines = [line.split('\t') for line in f]
            if self.shuffle:
                shuffle(lines)
                
        return lines
    
    def stupid_cycle(self, values, dct, number) -> dict: #util для семплинга
        
        dict_filter = {}
        
        for value in values:
            i = 0
            for k, v in dct.items():
                if v == value:
                    if i < number:
                        dict_filter[k] = v
                        i+=1
                
        return dict_filter
                
    def stupid_sampler(self) -> dict: #семплинг данных
        
        sents = self.read()
        values = []
        values_train = []
        values_test = []
        sents_train = []
        sents_test = []

        for line in sents:
            part, value, sentence = line[0], line[1], line[2]
            if value not in values:
                values.append(value)
            if 5 < len(sentence.split()) < 40:
                if part == 'tr':
                    values_train.append(value)
                    sents_train.append(sentence)
                if part =='te' or part=='va':
                    values_test.append(value)
                    sents_test.append(sentence)

        train_dict = dict(zip(sents_train, values_train))  
        test_dict = dict(zip(sents_test, values_test))

        values = set(values)
        number_one = round(self.train_size/len(values))
        number_two = round(self.test_size/len(values))

        dict_filter_train = self.stupid_cycle(values, train_dict, number_one)
        dict_filter_test = self.stupid_cycle(values, test_dict, number_two)
        return dict_filter_train, dict_filter_test
    
    def permute(self, dct) -> dict:
        l = list(dct.items())
        shuffle(l)
        return dict(l)
        
    def writer(self) -> str: #пишем в файлы
        
        """
        Writes to a file
        """
        
        if re.search(r'(?<=\/)[a-zA-Z][a-zA-Z]_[a-zA-Z]+(?=_)', self.path)[0]:
            dataset = re.search(r'(?<=\/)[a-zA-Z][a-zA-Z]_[a-zA-Z]+(?=_)', self.path)[0]
            path = path_project+f'/large_data_{dataset}'
        else:
            path = path_project+'/large_data'
            
        if not os.path.isdir(path):
            os.mkdir(path)
            
        if not os.path.isdir(path+f'/data_{self.category}'):
            os.mkdir(path+f'/data_{self.category}')
        
        result_path_datatrain = path+f"/data_{self.category}/datatrain_{self.category}.txt"
        result_path_labeltrain = path+f"/data_{self.category}/labeltrain_{self.category}.txt"
        
        
        result_path_datatest = path+f"/data_{self.category}/datatest_{self.category}.txt"
        result_path_labeltest = path+f"/data_{self.category}/labeltest_{self.category}.txt"
        
        with open(result_path_datatrain, "w", encoding="utf-8") as traindata, \
             open(result_path_labeltrain, "w", encoding="utf-8") as trainlabel, \
             open(result_path_datatest, "w", encoding="utf-8") as testdata, \
             open(result_path_labeltest, "w", encoding="utf-8") as testlabel:
            
            
            dict_filter_train, dict_filter_test = self.stupid_sampler()
            if self.shuffle:
                dict_filter_train = self.permute(dict_filter_train)
                dict_filter_test = self.permute(dict_filter_test)
    
            
            for sentence, value in dict_filter_train.items():
                traindata.writelines(sentence)
                trainlabel.writelines(value + '\n')


            for sentence, value in dict_filter_test.items():
                testdata.writelines(sentence)
                testlabel.writelines(value + '\n')
                                                                  
        
        return result_path_datatrain, result_path_labeltrain, result_path_datatest, result_path_labeltest

In [8]:
class Data:
    
    def __init__(self, path_trdata, path_tedata):
        
        self.path_trdata = path_trdata
        self.path_trlabel = path_trlabel
        self.path_tedata = path_tedata
        self.path_telabel = path_telabel
        self.category = re.search(r'(?<=_)[a-zA-Z]+(?=.txt)', path_trdata)[0]
        self.dataset = re.search(r'(?<=_)[a-zA-Z]+_[a-zA-Z]+(?=\/)', path_trdata)[0]
        
    def jsons(self, model):
        
        path = path_project+f'/large_data_{self.dataset}/data_{self.category}'
        
        
        transformers_extractor.extract_representations(model,
        self.path_trdata,
        path+'/activations_train.json',
        aggregation="average" #last, first
        )
        
        transformers_extractor.extract_representations(model,
        self.path_tedata,
        path+'/activations_te.json',
        aggregation="average" #last, first
        )

In [10]:
files = []
for dirname, _, filenames in os.walk(path_project+'/data'):
    for filename in filenames: #обходим все файлы-категории
        file = os.path.join(dirname, filename)
        if not re.search(r'/data/.gitignore', file):
            files.append(file)

In [9]:
for file in files:
    splitter = ConvertSample(file)
    
    #получаем трейновую и тестовую выборку
    path_trdata, path_trlabel, path_tedata, path_telabel = splitter.writer()
    
    #получаем эмбеддинги
    data = Data(path_trdata, path_tedata)
    data.jsons('bert-base-uncased')
    clear_output(wait=False)